In [9]:
print("Model Training")

Model Training


In [2]:
pwd

'g:\\RealProject\\End_to_End\\research'

In [3]:
import os
os.chdir("../")

In [4]:
pwd

'g:\\RealProject\\End_to_End'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    targer_column: str

In [6]:
from src.datascience.constant import *

In [7]:
class ConfigurationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer 
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet
        

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            targer_column = schema.name
        )
        return model_trainer_config


In [8]:
import pandas as pd
import os 
from src.datascience import logger
import joblib
from sklearn.linear_model import ElasticNet

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_X = train_data.drop([self.config.targer_column], axis=1)
        train_y = train_data[[self.config.targer_column]]
        train_X = test_data.drop([self.config.targer_column], axis=1)
        train_y = test_data[[self.config.targer_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state= 42)
        lr.fit(train_X,train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
